In [1]:
import pickle
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import pandas as pd
import random
import time
import json
from collections import Counter
import os
import copy
import seaborn as sns

In [3]:
# def get_all_filenames(base='./data/caida', upper_bound=0):
#     dir_list = [base]
#     ret_ll = []

#     cnt = 0

#     # go through all directory on the `base`
#     while dir_list:
#         cur_dir = dir_list.pop(0)
#         cur_list = os.listdir(cur_dir)
#         # go through all directory on a directory
#         for cur in cur_list:            
#             path = cur_dir + '/' + cur
#             # is a pcap file, run pcapParser
#             if (os.path.isfile(path)):
#                 ret_ll.append(path)
#                 cnt += 1
#             # isdirectory, push to list
#             else:
#                 dir_list.append(path)
#             # number of files processed is below the upper bound
#             if upper_bound != 0 and cnt >= upper_bound:
#                 print(f'files count: {cnt}')
#                 return ret_ll
#     print(f'files count: {cnt}')
#     return ret_ll

In [4]:
# base = './data/caida'
# filename_list = get_all_filenames(base, 1)

In [5]:
# filename_list = ['./data/caida/20181018/60s/equinix-nyc.dirA.20181018-130900.UTC.anon.csv', ]

In [2]:
### read file

filename = './data/caida/20181018/60s/equinix-nyc.dirA.20181018-130900.UTC.anon.csv'
# flowkeys = ['srcip', 'dstip', 'srcport', 'dstport', 'proto']
# flowkeys = ['srcip', 'srcport']
flowkeys = ['srcip']
# flowkeys = ['dstip', 'dstport']
start = time.time()
df = pd.read_csv(filename)
end = time.time()
print(end - start)
# df.groupby(flowkeys).size()
df

51.5672402381897


,srcip,dstip,srcport,dstport,proto,time,pkt_len,version,ihl,tos,id,flag,off,ttl,chksum
0,2835347327,0,21974,0,NaN,1539868140000000,0,6,0,0,85,0,4410,217,57984
1,1685889326,3571699120,443,47268,TCP,1539868140000002,1452,4,5,0,32226,2,0,57,5675
2,2258941833,3978971486,2515,56488,TCP,1539868140000003,52,4,5,0,27666,2,0,54,25595
3,3542516037,3608343929,443,39216,TCP,1539868140000006,1400,4,5,32,32741,0,0,120,40066
4,2353775117,3569582299,443,20744,TCP,1539868140000008,204,4,5,0,53674,2,0,59,51593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37781421,1311461803,3571699030,56962,49221,UDP,1539868199999994,48,4,5,0,5923,0,0,109,62601
37781422,3306889401,3447550259,80,26276,TCP,1539868199999994,1500,4,5,0,50136,2,0,61,20414
37781423,385030285,3450273070,443,55231,TCP,1539868199999996,52,4,5,8,61462,2,0,47,24912
37781424,1055819853,3231989353,5223,38011,TCP,1539868199999998,163,4,5,8,22792,2,0,46,11516


In [27]:
# define ip address <--> int converting function
import socket
import struct
def ip2long(ip):
    """
    Convert an IP string to long
    """
    packedIP = socket.inet_aton(ip)
    return struct.unpack("!L", packedIP)[0]

def long2ip(val):
    return socket.inet_ntoa(struct.pack('!L', val))

In [70]:
total_num_of_pkt = len(df)
total_num_of_flow = len(df.groupby(flowkeys).size())
# total_num_of_pkt / total_num_of_flow
np.argmax(df.groupby(flowkeys).size().values)
df.groupby(flowkeys).size().index[298030]
np.max(df.groupby(flowkeys).size().values)

1166273

In [87]:
print('original total_num_of_flow',total_num_of_flow)
s = np.random.zipf(a=1.1, size=total_num_of_flow)
s2 = []
for d in s:
    if d < 1166273/10:
        s2.append(d)
print(f'max: {max(s2):,}')
print(f'my total_num_of_pkt: {sum(s2):,}')
print(f'original total_num_of_pkt: {total_num_of_pkt:,}')

ratio = sum(s2) / total_num_of_pkt
print(f'ratio: {ratio}')
s2 /= ratio
s2 = s2.astype(int)
s2 += 1
print(f'max: {max(s2):,}')
print(f'my total_num_of_pkt: {sum(s2):,}')

original total_num_of_flow 421828
max: 116,611
my total_num_of_pkt: 1,612,175,287
original total_num_of_pkt: 37,781,426
ratio: 42.671107411350754
max: 2,733
my total_num_of_pkt: 37,976,784


In [7]:
# flowkey & packet size mapping
flowkey_to_pktsize = df.groupby(flowkeys).size()
print(flowkey_to_pktsize)
# how many flowkeys
flowkeys_count = len(df.groupby(flowkeys).size())
print('\nflowkeys_count:', flowkeys_count)

srcip
2433           1
391268         1
391274         2
391286        30
444372         2
              ..
4294952950     1
4294953882     1
4294953981     1
4294954493     1
4294964991     1
Length: 421828, dtype: int64

flowkeys_count: 421828


In [12]:
np.max(df.groupby(flowkeys).size().values)

1166273

## Sampling followed distributions

In [365]:
# howm many sample times for distribution
# it controls the how many flowkeys in dataset (upper bound)
sample_times = 170000
target_packet_cnt = int(15e6)

## https://numpy.org/doc/1.16/reference/routines.random.html

In [700]:
def get_zipf_count(sample_times, zipf_alpha=1.7, target_packet_cnt = int(15e6)):

    s = np.random.zipf(a=zipf_alpha, size=sample_times)
    # print('sum:', f'{s.sum():,}')
    cnt = 0
    tmp = []
    for idx, d in enumerate(s):
        # avoid 1 flowkey has too many packets (too large)
        if d < target_packet_cnt/5:
            cnt += d
            tmp.append(d)
            if cnt > target_packet_cnt:
                print('flowkeys_cnt:', idx, f'total_packet_cnt: {cnt:,}')
                break
    s = tmp
    

    bin_count = np.bincount(s)[1:]
    print('zipf:', bin_count[:10], f'{len(bin_count):,}')
    
    x = np.arange(1, len(bin_count) + 1)
    
    return x, bin_count
    
zipf_x, zipf_count = get_zipf_count(sample_times)
print(f'{len(zipf_x):,}')
# plt.bar(zipf_x, zipf_count, alpha=0.5, label='sample count')

idx: 74888 packet_cnt: 15,082,401
[36311 11360  5612  3419  2350  1723  1362  1051   877   700] 2,660,236
2,660,236


In [702]:
def get_uniform_count(sample_times, high_bound = 1e3, interval = 100, target_packet_cnt = int(15e6)):
    s = np.random.uniform(1, high_bound, sample_times)
    s = s.astype(int)
    # print('sum:', f'{s.sum():,}')
    cnt = 0
    tmp = []

    for idx, d in enumerate(s):
        # sampling interval, e.g. 100, 200, 300....
        d *= interval
        # avoid 1 flowkey has too many packets (too large)
        if d < target_packet_cnt / 5:
            cnt += d
            tmp.append(d)
            if cnt > target_packet_cnt:
                print('flowkeys_cnt:', idx, f'total_packet_cnt: {cnt:,}')
                break
    s = tmp
    

    bin_count = np.bincount(s)
    print('uniform:', bin_count[:10], f'{len(bin_count):,}')
    # a = plt.hist(s, density=True)
    
    x = np.arange(1, len(bin_count) + 1)
    return x, bin_count
uniform_x, uniform_count = get_uniform_count(sample_times)
print(len(uniform_x))

idx: 303 packet_cnt: 15,002,700
[0 0 0 0 0 0 0 0 0 0] 99801
99801


In [575]:
# def get_lognormal_count(sample_times, mu=8, sigma=1):
#     mu, sigma = 8, 1 # mean and standard deviation
#     s = np.random.lognormal(mu, sigma, sample_times)
#     print(s)
#     s = s.astype(int)
#     bin_count = np.bincount(s)
#     print(bin_count)
#     count, bins, ignored = plt.hist(s, 100, density=True, align='mid')
# get_lognormal_count(10000)

In [484]:
# mu, sigma = 0, 100000 # mean and standard deviation
# s = np.random.normal(mu, sigma, sample_times)
# count, bins, ignored = plt.hist(s + 6e5, 100, density=True)
# # s.astype(int) + 1000

## Get real count

In [8]:
vals = df.groupby(flowkeys).size().values
def get_real_count(vals):
    dist_dict = dict(Counter(vals))
    dist_dict = {k: v for k, v in sorted(dist_dict.items(), key = lambda x: x[0])}
    x = dist_dict.keys()
    x = list(x)

    count = np.asarray(list(dist_dict.values()), dtype=int) # / float(sum(dist_dict.values()))
    print(x[-10:], count[-10:])
    
    return x, count

real_x, real_count = get_real_count(vals)

[197352, 203200, 325820, 331460, 334424, 348367, 428137, 470708, 536081, 1166273] [1 1 1 1 1 1 1 1 1 1]


In [10]:
def get_probability_and_difference(exp_count):
    pktsize_to_probability = {}
    # calculate the difference between real & expect
    diff_count = copy.deepcopy(exp_count)
    for idx, cnt in zip(real_x, real_count):
        try:
            exp_cnt = exp_count[idx - 1]
            diff_count[idx - 1] -= cnt
        except:
            exp_cnt = 0

        # flowkey's keeping probability
        prob = float(exp_cnt) / cnt
        # if prob > 1:
        #     print(exp_cnt / cnt)
        pktsize_to_probability[idx] = prob
        # print(idx, cnt, prob)
    print(len(exp_count), len(pktsize_to_probability))
    return diff_count, pktsize_to_probability
diff_count, pktsize_to_probability = get_probability_and_difference(zipf_count)

473324 4142


In [60]:
### Drop flowkeys which more than distribution

def split_flowkeys(df, flowkey_to_pktsize):
    cnt = 0
    drop_cnt = 0
    df2 = df.copy(deep=True)

    remaining_keys = []
    removed_keys = []
    start = time.time()
    for key in list(df.groupby(flowkeys).size().index):
        pktsize = flowkey_to_pktsize[key]
        prob = pktsize_to_probability[pktsize]
        # drop this flowkey
        if random.random() > prob:
            drop_cnt += 1
            removed_keys.append(key)
        # keep this key
        else:
            remaining_keys.append(key)
        cnt += 1
        if cnt % 100000 == 0:
            print(time.time())

    end = time.time()
    print(end - start)
    print('total cnt:', cnt, 'drop cnt:', drop_cnt)
    
    df2 = df2.loc[df2['srcip'].isin(remaining_keys)]
    
    return remaining_keys, removed_keys, df2
remaining_keys, removed_keys, df_remaining_keys = split_flowkeys(df, flowkey_to_pktsize)

1681463732.3426695
1681463732.6170301
1681463732.8923442
1681463733.169919
2.2816250324249268
total cnt: 421828 drop cnt: 221947


In [64]:
print(len(remaining_keys), len(removed_keys))
print('df_remaining_keys.shape:', df_remaining_keys.shape)
print('df.shape:', df.shape)

199881 221947
df_remaining_keys.shape: (1504423, 15)
df.shape: (37781426, 15)


In [63]:
# define ip address <--> int converting function
import socket
def ip2long(ip):
    """
    Convert an IP string to long
    """
    packedIP = socket.inet_aton(ip)
    return struct.unpack("!L", packedIP)[0]

def long2ip(val):
    return socket.inet_ntoa(struct.pack('!L', val))

In [67]:
# create dummy packet from removed key

def create_dummy_packet(diff_count, removed_keys, df):
    dummy_pkt_cnt = 0
    idx_removed_key = 0
    pkts_list = []

    # `i` is index in `diff_count`, `i + 1` is also the packet size
    for i, diff in enumerate(diff_count):
        # it needs to create dummy packet
        if diff > 0:
            # print(i, d)
            pktsize = i + 1
            dummy_pkt_cnt += pktsize * diff
            # print(pktsize)
            # how many flowkey should be created on this packet size
            for num in range(diff):
                pkt_info = df[df['srcip'] == removed_keys[idx_removed_key]].iloc[0]
                tmp_ll = []
                # TCP
                if pkt_info['proto'] == 'TCP':
                    pkt = Ether(dst='00:11:22:33:44:55')/IP(src=long2ip(pkt_info['srcip']), dst=long2ip(pkt_info['dstip']))/TCP(sport=pkt_info['srcport'], dport=pkt_info['dstport'])
                # UDP
                else:
                    pkt = Ether(dst='00:11:22:33:44:55')/IP(src=long2ip(pkt_info['srcip']), dst=long2ip(pkt_info['dstip']))/UDP(sport=pkt_info['srcport'], dport=pkt_info['dstport'])

                # setup timestamp
                pkt.time = pkt_info.time / 1e6
                # duplicate packets, according to pktsize
                tmp_ll = [pkt] * pktsize
                pkts_list += tmp_ll
                idx_removed_key += 1

    print(dummy_pkt_cnt)
    print(idx_removed_key)
    return pkts_list
pkts_list = create_dummy_packet(diff_count, removed_keys, df)
print('len(pkts_list):', len(pkts_list))

2049459
123


In [70]:
global_cnt = 0
global_pkt_of_remaining_key = []

# create packet for remaining flowkeys
def create_packet(pkt_info):
    global global_cnt
    global global_pkt_of_remaining_key
    
    global_cnt += 1
    if global_cnt % 100000 == 0:
        print(global_cnt)

    if pkt_info['proto'] == 'TCP':
        pkt = Ether(dst='00:11:22:33:44:55')/IP(src=long2ip(pkt_info['srcip']), dst=long2ip(pkt_info['dstip']))/TCP(sport=pkt_info['srcport'], dport=pkt_info['dstport'])
    # UDP
    else:
        pkt = Ether(dst='00:11:22:33:44:55')/IP(src=long2ip(pkt_info['srcip']), dst=long2ip(pkt_info['dstip']))/UDP(sport=pkt_info['srcport'], dport=pkt_info['dstport'])

    # setup timestamp
    pkt.time = pkt_info.time / 1e6
    global_pkt_of_remaining_key.append(pkt)

In [72]:
start = time.time()

# create packets for remaining keys
df_remaining_keys.apply(create_packet, axis=1)

end = time.time()
print(end - start, 'sec')

assert(len(global_pkt_of_remaining_key) == global_cnt)
print(len(global_pkt_of_remaining_key))

0


In [436]:
# concatenate packets of remaining keys & dummy keys, svae into pcap files
start = time.time()

ll = global_pkt_of_remaining_key + pkts_list
print(len(ll))
name = '20181018-130900_srcip_zipf'
wrpcap(name + '.pcap', ll)

end = time.time()
print(end - start, 'sec')

4784199
1433.309710741043


In [68]:
# global_cnt
# df2

In [19]:
df[df['srcip']==1838263254]

,srcip,dstip,srcport,dstport,proto,time,pkt_len,version,ihl,tos,id,flag,off,ttl,chksum
10,1838263254,3347579298,443,47268,TCP,1539868140000018,56,4,5,8,12369,0,0,54,29146


In [61]:
# df_remaining_keys.to_csv('df2.csv', sep=',', index=False, encoding='utf-8')

In [73]:
# start = time.time()

# df.to_csv('perf.csv')

# end = time.time()
# print(end - start)

## Create packet (pcap)

In [21]:
from scapy.all import *

In [35]:
pkts = []
# pkt = Ether()/IP(dst='8.8.8.8')/TCP(dport=53,flags='S')
# pkts.append(pkt)
# pkt = Ether()/IP(src='1.1.1.1', dst='8.8.8.8')/TCP(dport=53,flags='S')
# pkts.append(pkt)

src_ip = "192.168.0.1"
dst_ip = "10.0.0.1"
src_port = 12345
dst_port = 80
# Define the packet flags
syn_flag = 0x02
ack_flag = 0x10

pkt = Ether(dst='00:11:22:33:44:55')/IP(src=src_ip, dst=dst_ip)/TCP(sport=src_port, dport=dst_port)
pkts.append(pkt)

pkt = Ether(dst='00:11:22:33:44:55')/IP(src=src_ip, dst=dst_ip)/UDP(sport=src_port, dport=dst_port)
pkts.append(pkt)

timestamp = 1539868199.123456

pkt = Ether(dst='00:11:22:33:44:55')/IP(src=src_ip, dst=dst_ip)/UDP(sport=src_port, dport=dst_port)
pkt.time = timestamp
pkts.append(pkt)

ttl_value = 100
packet_length = 200

# pkt = Ether(dst='00:11:22:33:44:55')/IP(src=src_ip, dst=dst_ip, ttl=ttl_value)/UDP(sport=src_port, dport=dst_port)/Raw(load=b"A" * packet_length)
# pkts.append(pkt)

wrpcap('foo.pcap', pkts)

In [36]:
pkts = rdpcap("foo.pcap")
for p in pkts:
    print(p)

Ether / IP / TCP 192.168.0.1:12345 > 10.0.0.1:http S
Ether / IP / UDP 192.168.0.1:12345 > 10.0.0.1:80
Ether / IP / UDP 192.168.0.1:12345 > 10.0.0.1:80


In [79]:
# pkts = rdpcap("equinix-nyc.dirA.20180517-130900.UTC.anon.pcap")
# pcap_name = '20181018-130900_srcip_zipf.csv'
pcap_name = '20181220-131100-srcip-zipf.csv'
pkt_df = pd.read_csv(pcap_name)

In [80]:
pkt_df

,srcip,dstip,srcport,dstport,proto,time,pkt_len,version,ihl,tos,id,flag,off,ttl,chksum
0,3466822664,3257345458,1126,3393,TCP,1545311460000001,40,4,5,0,1,0,0,64,16458
1,3885270467,3449052658,443,39048,TCP,1545311460000089,40,4,5,0,1,0,0,64,57840
2,3307969487,3978971499,47646,443,TCP,1545311460000134,40,4,5,0,1,0,0,64,58174
3,1706519271,2340553942,443,52658,TCP,1545311460000173,40,4,5,0,1,0,0,64,6617
4,2283260735,3450146404,29273,40689,TCP,1545311460000257,40,4,5,0,1,0,0,64,19311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1559829,33718550,4225769677,443,52220,TCP,1545311472147219,40,4,5,0,1,0,0,64,56073
1559830,33718550,4225769677,443,52220,TCP,1545311472147219,40,4,5,0,1,0,0,64,56073
1559831,33718550,4225769677,443,52220,TCP,1545311472147219,40,4,5,0,1,0,0,64,56073
1559832,33718550,4225769677,443,52220,TCP,1545311472147219,40,4,5,0,1,0,0,64,56073
